<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

In [3]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/chisel-bootcamp/source/load-ivy.sc"

In [4]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

import dotvisualizer._

In [6]:
class TEAIO extends Bundle {
  val v0  = Input(UInt(32.W))
  val v1  = Input(UInt(32.W))
  val k0  = Input(UInt(32.W))
  val k1  = Input(UInt(32.W))
  val k2  = Input(UInt(32.W))
  val k3  = Input(UInt(32.W))
  val start = Input(Bool())   // control: begin encryption
  val out0  = Output(UInt(32.W))
  val out1  = Output(UInt(32.W))
  val done  = Output(Bool())  // control: finished
}

defined class TEAIO

In [7]:
class TEAEncrypt extends Module {
  val io = IO(new TEAIO)

  val Delta = "h9E3779B9".U(32.W)
  val Rounds = 32.U

  val v0 = Reg(UInt(32.W))
  val v1 = Reg(UInt(32.W))
  val sum = RegInit(0.U(32.W))
  val round = RegInit(0.U(6.W))
  val busy = RegInit(false.B)

  io.out0 := v0
  io.out1 := v1
  io.done := !busy && (round === Rounds)

  when (io.start && !busy) {
    v0 := io.v0
    v1 := io.v1
    sum := 0.U
    round := 0.U
    busy := true.B
  } .elsewhen (busy) {
    val newV0 = v0 + (((v1 << 4) + io.k0) ^ (v1 + sum) ^ ((v1 >> 5) + io.k1))
    val newSum = sum + Delta
    val newV1 = v1 + (((newV0 << 4) + io.k2) ^ (newV0 + newSum) ^ ((newV0 >> 5) + io.k3))

    v0 := newV0
    v1 := newV1
    sum := newSum
    round := round + 1.U

    when (round === (Rounds - 1.U)) {
      busy := false.B
    }
  }
}

defined class TEAEncrypt

In [8]:
test(new TEAEncrypt()) { c =>
  val key = Array(0x00010203, 0x04050607, 0x08090a0b, 0x0c0d0e0f)
  val v0_in = 0x41424344  // "ABCD"
  val v1_in = 0x45464748  // "EFGH"

  c.io.v0.poke(v0_in.U)
  c.io.v1.poke(v1_in.U)
  c.io.k0.poke(key(0).U)
  c.io.k1.poke(key(1).U)
  c.io.k2.poke(key(2).U)
  c.io.k3.poke(key(3).U)

  c.io.start.poke(true.B)
  c.clock.step(1)
  c.io.start.poke(false.B)

  c.clock.step(32)
  c.io.done.expect(true.B)

  println(f"Cipher = ${c.io.out0.peek().litValue}%08x ${c.io.out1.peek().litValue}%08x")
}
println("SUCCESS!!")

Elaborating design...
Done elaborating.
Cipher = 839d8fbb ecf9c9db
test TEAEncrypt Success: 0 tests passed in 35 cycles in 0.090992 seconds 384.65 Hz
SUCCESS!!
